data was scrapped on june 13 2020

In [1]:
#Import Libraries
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


## Import and Cleaning

In [2]:
calendar = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/calendar.csv.gz');
listings1 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/listings.csv');
listings2 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/listings.csv.gz');
neighborhoods = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/neighbourhoods.csv');
reviews1 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/reviews.csv');
reviews2 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/reviews.csv.gz');

#Data Notes:

#locations are not exact (0-450 feet from actual address) due to how airbnb displays data. Therefore a heatmap will work well
#price is in dollars

c:\users\alex cerce\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (43,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Start by fixing the data types for each table

In [3]:
#Data cleaning functions used below

#I would prefer integers to have no trailing zeros

#This function will search a specifc dataframe column and return the number of non integer values and NAN values
#I am doing this because I want to keep the trailing zeros if there are values in the array 
#I wont be able to take away trailing zeros if there are any nan values
def find_noninteger(Dataframe,feature): #feature should be input as a string to be consistent with normal pandas format
    value=Dataframe[feature];
    float_count=0;na_count=0;index_nonint=[];index_na=[];
    for i in range(len(value)):
        if (value[i]).is_integer()==0 | pd.isna(value[i])==0:
            float_count +=1
            index_nonint.append(i)
        if pd.isna(value[i])==1:
            na_count +=1
            #index_na.append(i)
    if float_count==1 and na_count==1:
        print('There is',float_count,'non integer value and',na_count,'NAN value')
    elif float_count==1 and na_count!=1:
        print('There is',float_count,'non integer value and',na_count,'NAN values')
    elif float_count!=1 and na_count==1:
        print('There are',float_count,'non integer values and',na_count,'NAN value')
    elif float_count!=1 and na_count!=1:
        print('There are',float_count,'non integer values and',na_count,'NAN values')
    if index_nonint or index_na:
        print('The non integer values are located at:',index_nonint)
'''
#remove dollar sign from price
def remove_dollarsign(Dataframe,feature):
    feature=Dataframe[feature];
    for i in range(len(feature)):
        if isinstance(feature[i],str)==1 and feature[i][0]=='$':
            feature[i]=feature[i][1:]''';

### Calendar Data Table

In [4]:
calendar.head(n=5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,80843,2020-06-14,f,$39.00,$39.00,2.0,1125.0
1,186851,2020-06-15,f,$33.00,$33.00,62.0,300.0
2,186851,2020-06-16,f,$33.00,$33.00,62.0,300.0
3,186851,2020-06-17,f,$33.00,$33.00,62.0,300.0
4,186851,2020-06-18,f,$33.00,$33.00,62.0,300.0


In [5]:
#Missing Values


In [6]:
#check columns for decimal values. This will indicated whether or not I should remove trailing zeros
print('minimum nights:')
find_noninteger(calendar,'minimum_nights')
print('maximum nights:')
find_noninteger(calendar,'maximum_nights')
print('-'*50);print('Before:');print(calendar.dtypes)  #This indicates, that once I deal with missing values I can convert these cols to int


#Convert Data Types
#convert to datetime
calendar['date']=calendar['date'].astype('datetime64')
#convert to string dtype and capitalize T/F values
calendar['available']=calendar['available'].str.replace('f','F').str.replace('t','T').astype(str)
#remove dollar signs, remove comas, and convert to float
calendar['price']=calendar['price'].str.replace('$','').str.replace(',','').astype('float64') 
calendar['adjusted_price']=calendar['adjusted_price'].str.replace('$','').str.replace(',','').astype('float64')
#cant convert final two cols to integer yet due to missing values
print('-'*50);print('After:')
print(calendar.dtypes)


minimum nights:
There are 0 non integer values and 141 NAN values
maximum nights:
There are 0 non integer values and 141 NAN values
--------------------------------------------------
Before:
listing_id          int64
date               object
available          object
price              object
adjusted_price     object
minimum_nights    float64
maximum_nights    float64
dtype: object
--------------------------------------------------
After:
listing_id                 int64
date              datetime64[ns]
available                 object
price                    float64
adjusted_price           float64
minimum_nights           float64
maximum_nights           float64
dtype: object


In [7]:
calendar.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,80843,2020-06-14,F,39.0,39.0,2.0,1125.0
1,186851,2020-06-15,F,33.0,33.0,62.0,300.0
2,186851,2020-06-16,F,33.0,33.0,62.0,300.0
3,186851,2020-06-17,F,33.0,33.0,62.0,300.0
4,186851,2020-06-18,F,33.0,33.0,62.0,300.0


### Listings_1 Data Table

In [8]:
listings1.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,2217,Ion,Mitte,Brunnenstr. Süd,52.53454,13.40256,Entire home/apt,60,4,132,2020-04-03,2.59,7,180
1,3176,Fabulous Flat in great Location,3718,Britta,Pankow,Prenzlauer Berg Südwest,52.53500,13.41758,Entire home/apt,90,62,146,2020-05-27,1.09,1,0
2,3309,BerlinSpot Schöneberg near KaDeWe,4108,Jana,Tempelhof - Schöneberg,Schöneberg-Nord,52.49885,13.34906,Private room,28,7,27,2019-05-31,0.32,1,364
3,7071,BrightRoom with sunny greenview!,17391,BrightRoom,Pankow,Helmholtzplatz,52.54316,13.41509,Private room,33,1,293,2020-03-31,2.22,2,61
4,9991,Geourgeous flat - outstanding views,33852,Philipp,Pankow,Prenzlauer Berg Südwest,52.53303,13.41605,Entire home/apt,180,6,8,2020-01-04,0.14,1,10


In [ ]:
calendar['minimum_nights']=calendar['minimum_nights'].astype(int,errors='ignore')
calendar['maximum_nights']=calendar['maximum_nights'].astype(int,errors='ignore')

ball=pd.DataFrame(data={'night':[1.0,1.2,np.nan,1.00,2.0]})

ball['night']=ball['night'].astype(int,errors='ignore')

print(ball['night'])